In [1]:
import numpy as np
import pandas as pd

from itertools import product
from collections import Counter
from datasets import load_dataset
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score

In [2]:
def metric_calculation(pred, gt):    
    acc=accuracy_score(gt, pred)
    f1=f1_score(gt, pred, average='macro')
    confusion=confusion_matrix(gt, pred)
    fpr=confusion[0,1]/len(gt) ## predict to be 1; actual 0
    fnr=confusion[1,0]/len(gt) ## predict to be 0; actual 1
    return acc, f1, fpr, fnr
    
def post_processing(pred):
    new_pred=[]
    for i in pred:
        i=i.lower()
        if 'response' in i:
            try: new_pred.append(i.split('response')[1].split()[1].replace('</s>', ''))
            except: new_pred.append(2)
        elif 'output' in i:
            try: new_pred.append(i.split('output')[1].split()[1].replace('</s>', ''))
            except: new_pred.append(2)
        else:
            try: new_pred.append(i.split()[0].replace('</s>', ''))
            except:new_pred.append(2)
    new_pred = np.array([int(float(i)) if i in ['0', '0.0', '1', '1.0'] else 2 for i in new_pred])
    return new_pred

In [3]:
ds = load_dataset("beanham/spatial_union_dataset")
test=ds['test']
gt=np.array(test['label'])

In [16]:
results=[]
metric_values = ['random', 'worst_single', 'best_single', 'worst_comb', 'best_comb']
for model in ['4o_mini', '4o','qwen_plus']:
    for value in metric_values:
        few_shot_hints=np.load(f'base/{model}_correction/{model}_{value}_few_shot_with_heur_hint_angle_area_correction.npy')
        #few_shot_values=np.load(f'base/{model}_correction/{model}_{value}_few_shot_with_heur_value_angle_area_correction.npy')
        few_shot_hints=post_processing(few_shot_hints)
        #few_shot_values=post_processing(few_shot_values)
        few_hints_metrics=metric_calculation(few_shot_hints, gt)
        #few_values_metrics=metric_calculation(few_shot_values, gt)
        results.append([model, value, 'few_shot_hints', round(few_hints_metrics[0],3), few_hints_metrics[1]])
        #results.append([model, value, 'few_shot_values', round(few_values_metrics[0],3), few_values_metrics[1]])
results=pd.DataFrame(results, columns=['model', 'value', 'prompt', 'acc', 'f1'])

In [17]:
results

,model,value,prompt,acc,f1
0,4o_mini,random,few_shot_hints,0.363,0.269589
1,4o_mini,worst_single,few_shot_hints,0.366,0.273956
2,4o_mini,best_single,few_shot_hints,0.363,0.272567
3,4o_mini,worst_comb,few_shot_hints,0.363,0.269589
4,4o_mini,best_comb,few_shot_hints,0.366,0.273956
5,4o,random,few_shot_hints,0.657,0.654942
6,4o,worst_single,few_shot_hints,0.867,0.857486
7,4o,best_single,few_shot_hints,0.900,0.895440
8,4o,worst_comb,few_shot_hints,0.857,0.854272
9,4o,best_comb,few_shot_hints,0.907,0.903584
